## Apriori algorithm

Apriori is an algorithm used for Association Rule Mining.

It searches for a series of frequent sets of items in the datasets.

It builds on associations and correlations between the itemsets.

It is the algorithm behind **"You may also like"** where you commonly saw in recommendation platforms.

In [1]:
import sys
from collections import defaultdict
from operator import itemgetter

import pandas as pd

## Convert Unix Timestamp to Datetime

In the expression `pandas.to_datetime(all_ratings['Datetime'], unit='s')`, the argument `unit='s'` specifies that the values in the 'Datetime' column of the `all_ratings` dataframe are Unix timestamps, which are expressed in **seconds since January 1, 1970** (also known as the **Unix epoch**). 

So, the letter **"s"** here stands for "seconds", indicating that the values in the 'Datetime' column are provided in seconds. By passing this argument to `pandas.to_datetime()`, the function will recognize the input as Unix timestamps and convert them to pandas datetime objects. 

Overall, the expression `pandas.to_datetime(all_ratings['Datetime'], unit='s')` is used to **convert the 'Datetime' column** of the `all_ratings` dataframe from Unix timestamps to **datetime objects**, which can be more easily manipulated and analyzed in pandas.

In [2]:
columns = ["UserID", "MovieID", "Rating", "Datetime"]

all_ratings = pd.read_csv('ml-100k/u.data', delimiter="\t", header=None, names=columns)

all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'], unit='s')

all_ratings.head(7)
# all_ratings.shape  # (100000, 4)

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16
5,298,474,4,1998-01-07 14:20:06
6,115,265,2,1997-12-03 17:51:28


In [3]:
all_ratings.Rating.unique()

# So it's like the star rating

array([3, 1, 2, 4, 5])

In [4]:
# Not all reviews are favorable! Our goal is "other recommended movies", 
# so we only want favorable reviews

all_ratings["Favorable"] = all_ratings["Rating"] > 3  # Creates a boolean column

all_ratings[10:15]

,UserID,MovieID,Rating,Datetime,Favorable
10,62,257,2,1997-11-12 22:07:14,False
11,286,1014,5,1997-11-17 15:38:45,True
12,200,222,5,1997-10-05 09:05:40,True
13,210,40,3,1998-03-27 21:59:54,False
14,224,29,3,1998-02-21 23:40:57,False


In [5]:
all_ratings[all_ratings["UserID"] == 1][:5]  # Column userid 1, 5 rows

,UserID,MovieID,Rating,Datetime,Favorable
202,1,61,4,1997-11-03 07:33:40,True
305,1,189,3,1998-03-01 06:15:28,False
333,1,33,4,1997-11-03 07:38:19,True
334,1,160,4,1997-09-24 03:42:27,True
478,1,20,4,1998-02-14 04:51:23,True


In [6]:
# Sample the dataset. You can try increasing the size of the sample,
# but the run time will be considerably longer

ratings = all_ratings[all_ratings['UserID'].isin(range(200))]

ratings.head(7)
# ratings  # 19531 rows × 4 columns

,UserID,MovieID,Rating,Datetime,Favorable
0,196,242,3,1997-12-04 15:55:49,False
1,186,302,3,1998-04-04 19:22:22,False
2,22,377,1,1997-11-07 07:18:36,False
4,166,346,1,1998-02-02 05:33:16,False
6,115,265,2,1997-12-03 17:51:28,False
9,6,86,3,1997-12-31 21:16:53,False
10,62,257,2,1997-11-12 22:07:14,False


In [7]:
# We start by creating a dataset of each user's favorable reviews
favorable_ratings = ratings[ratings["Favorable"]]

favorable_ratings[:5]

,UserID,MovieID,Rating,Datetime,Favorable
16,122,387,5,1997-11-11 17:47:39,True
20,119,392,4,1998-01-30 16:13:34,True
21,167,486,4,1998-04-16 14:54:12,True
26,38,95,5,1998-04-13 01:14:54,True
28,63,277,4,1997-10-01 23:10:01,True


## Create a dictionary

```py
favorable_reviews_by_users = dict((key, 
                                   frozenset(value.values)) for key, value in
                                  favorable_ratings.groupby("UserID")["MovieID"])
```

**keys:** user IDs

**values:** frozen sets of movie IDs that were favorably rated by that user

1. `favorable_ratings.groupby("UserID")["MovieID"]` **groups the rows** of the `favorable_ratings` dataframe by user ID and **selects** the MovieID **column.**
    * This gives a Series object where each element is a list of movie IDs that were favorably rated by a particular user.

2. `dict((k, frozenset(v.values)) for k, v in ...)` is a **dictionary comprehension** that iterates over each group of movie IDs and their associated user ID.
    * For each group, it creates a key-value pair in the dictionary.
    * The key is the user ID (`k`).
    * The value is a frozen set of movie IDs that were favorably rated by that user. 

3. `frozenset(v.values)` creates a frozen set from the list of movie IDs (`v.values`) associated with a particular user ID.
    * A frozen set is an immutable set that cannot be modified once it is created.
    * This is useful because it ensures that the set of movie IDs associated with each user ID cannot be accidentally modified later on.


In [8]:
# We are only interested in the reviewers who have more than one review

favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("UserID")["MovieID"])

len(favorable_reviews_by_users)

199

In [9]:
# Find out how many movies have favorable ratings

num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()

# Sort in descending order
num_favorable_by_movie.sort_values(by="Favorable", ascending=False)[:5]


,Favorable
MovieID,
50,100
100,89
258,83
181,79
174,74


### Create initial frequent itemset to work with items that have minimum support

In [10]:
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    """
    Find frequent itemsets
    """
    counts = defaultdict(int)

    for user, reviews in favorable_reviews_by_users.items():

        for itemset in k_1_itemsets:

            if itemset.issubset(reviews):

                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1

    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])


### Identify films with more than 50 favourable reviews

In [11]:
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 50  # cut-off

# k=1 candidates are the ISBNs with more than min_support favorable reviews
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                            for movie_id, row in num_favorable_by_movie.iterrows()
                            if row["Favorable"] > min_support)

print("\nThere are {} movies with more than {} favorable reviews\n".format(len(frequent_itemsets[1]), min_support))

sys.stdout.flush()

for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k - 1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        # print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets

# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]


There are 16 movies with more than 50 favorable reviews

I found 93 frequent itemsets of length 2
I found 295 frequent itemsets of length 3
I found 593 frequent itemsets of length 4
I found 785 frequent itemsets of length 5
I found 677 frequent itemsets of length 6
I found 373 frequent itemsets of length 7
I found 126 frequent itemsets of length 8
I found 24 frequent itemsets of length 9
I found 2 frequent itemsets of length 10
Did not find any frequent itemsets of length 11


In [12]:
print("\nFound a total of {0} frequent itemsets".format(
    sum(len(itemsets) 
        for itemsets in frequent_itemsets.values())))



Found a total of 2968 frequent itemsets


### Apriori algorithm identifies the frequent sets.

Lets use these to build our association rules.

In [13]:
# Now we create the association rules. First, they are candidates until the confidence has been tested
candidate_rules = []

for itemset_length, itemset_counts in frequent_itemsets.items():

    for itemset in itemset_counts.keys():

        for conclusion in itemset:  # iterate over each movie- conclusion
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))

print("\nThere are {} candidate rules".format(len(candidate_rules)))

print("\n", candidate_rules[:5])  # if the user recommends 50, they will also recommend 64



There are 15285 candidate rules

 [(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({50}), 1), (frozenset({1}), 50), (frozenset({1}), 56)]


In [14]:
# Now, we compute the confidence of each of these rules.
# Create dictionaries to store the number of times the premise leads to the conclusion
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)

for user, reviews in favorable_reviews_by_users.items():

    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule

        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(
    correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in candidate_rules}

# Choose only rules above a minimum confidence level
min_confidence = 0.8


In [15]:
# Filter out the rules with poor confidence
rule_confidence = {
    rule: confidence for rule, 
    confidence in rule_confidence.items() 
    if confidence > min_confidence
}

print(len(rule_confidence))


8428


In [16]:
# The operator item getter thingy

sorted_confidence = sorted(
    rule_confidence.items(), 
    key=itemgetter(1), 
    reverse=True
)


In [17]:
for index in range(5):
    print("\nRule #{0}".format(index + 1))

    (premise, conclusion) = sorted_confidence[index][0]

    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")



Rule #1
Rule: If a person recommends frozenset({98, 181}) they will also recommend 50
 - Confidence: 1.000


Rule #2
Rule: If a person recommends frozenset({172, 79}) they will also recommend 174
 - Confidence: 1.000


Rule #3
Rule: If a person recommends frozenset({258, 172}) they will also recommend 174
 - Confidence: 1.000


Rule #4
Rule: If a person recommends frozenset({1, 181, 7}) they will also recommend 50
 - Confidence: 1.000


Rule #5
Rule: If a person recommends frozenset({1, 172, 7}) they will also recommend 174
 - Confidence: 1.000



I need the titles, not the numbers...

## Get movie titles from dataset

In [18]:
movie_name_data = pd.read_csv('ml-100k/u.item', delimiter="|", header=None, encoding="mac-roman")

movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]


In [19]:
def get_movie_name(movie_id):
    """
    Get movie name
    """
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title


In [20]:
print(get_movie_name(1))
print(get_movie_name(2))
print(get_movie_name(3))
print(get_movie_name(4))
print(get_movie_name(5))

Toy Story (1995)
GoldenEye (1995)
Four Rooms (1995)
Get Shorty (1995)
Copycat (1995)


In [21]:
for index in range(5):
    print("\nRule #{0}".format(index + 1))

    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)

    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")



Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Confidence: 1.000


Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000


Rule #3
Rule: If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000


Rule #4
Rule: If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Confidence: 1.000


Rule #5
Rule: If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

